In [ ]:
import csv
import math
import subprocess

import pandas as pd
import matplotlib.pyplot as plt

# Trials

Let's run trials in Python from outside the Java source files.

## Setup file reading functions

In [ ]:
def read_baseline(baseline_filename):
    traversal_time = dict()
    with open(baseline_filename) as infile:
        isFirstLine = True
        vtime = dict()
        for s in infile:
            if isFirstLine:
                isFirstLine = False
            else:
                s2 = s.strip()
                d = s2.split(',')
                key = (d[3],d[4],d[5])
                if key in vtime:
                    t = abs(float(d[1])-float(vtime[key]))
                    traversal_time[key] = t
                else:
                    vtime[key] = d[1]
    return traversal_time

def read_delay(infilename, base_time):
    delay_time = dict()
    with open(infilename) as infile:
        isFirstLine = True
        vtime = dict()
        for s in infile:
            if isFirstLine:
                isFirstLine = False
            else:
                s2 = s.strip()
                d = s2.split(',')
                vin = d[0]
                t = d[1]
                key = (d[3],d[4],d[5])
                if vin in vtime:
                    traversal_time = abs(float(t)-float(vtime[vin]))
                    delay_time[vin] = traversal_time - base_time[key]
                else:
                    vtime[vin] = t
    return delay_time

def print_traversal_time(traversal_time):
    for (type,laneId,destRoad),t in traversal_time.items():
        print("[", type, ",", laneId, ",", destRoad, "] = ", t);

def avg_delay(delay_time):
	count_for_human = 0
	count_for_auto = 0
	delay_for_human = 0
	delay_for_auto = 0
	
	for vin,delay in delay_time.items():
		if (int(vin) >= 10000):
			count_for_human += 1
			delay_for_human += delay
		else:
			count_for_auto += 1
			delay_for_auto += delay

	if (count_for_human != 0):
	  avg_delay_human = delay_for_human / count_for_human
	else:
	  avg_delay_human = 0
	  
	if (count_for_auto != 0):
	  avg_delay_auto = delay_for_auto / count_for_auto
	else:
	  avg_delay_auto = 0

	if (count_for_human != 0 or count_for_auto != 0):
	  avg_delay = (delay_for_human + delay_for_auto) / (count_for_human + count_for_auto)
	else:
	  avg_delay = 0
	return [avg_delay, avg_delay_human, avg_delay_auto]

## Automate trials

In [ ]:
def hhv_trial(traffic_level, hhv_proportion, buffer_factor, baseline_filename='output/baseline.csv', n=1):
    '''
    Run a HHV trial
    
    n:   how many iterations of the same trial to run
    '''
    
    base_time = read_baseline(baseline_filename)

    writer = csv.writer(open("output/delay_result_for_human_t{}_p{}_b{}x.csv".format(traffic_level, hhv_proportion, buffer_factor), "w"))
    command = 'java src.main.java.aim4.Main_HHV_experiments -d {} {} {}'.format(traffic_level, hhv_proportion, buffer_factor)
    fileName = 'output/ts_hud_{}_{}_{}.csv'.format(traffic_level, hhv_proportion, buffer_factor)

    df = pd.DataFrame(columns=['avg_delay', 'delay_human', 'avg_delay_auto'])

    for i in range(n):
        print(command)
        subprocess.run(command, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        print(fileName)
        a = read_delay(fileName, base_time)
        avg_delay_time = avg_delay(a)
        df.loc[i] = avg_delay_time
        writer.writerow(avg_delay_time)
    subprocess.run("rm " + fileName)
    
    return df

df = hhv_trial(0.1, 0.1, 4)
df.head()